# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet

#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [2]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [3]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [4]:
# number of callbacks for white-sounding names
sum(data[data.race=='b'].call)

157.0

In [5]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


In [6]:
print('The number of observations in the dataset are', len(data))

The number of observations in the dataset are 4870


### Q1. What test is appropriate for this problem? Does CLT apply?

A two sample z-test is appropriate for this problem as there are two groups to test against each other.

As a general rule, sample sizes equal to or greater than 30 are considered sufficient for the CLT to hold, meaning the distribution of the sample means is fairly normally distributed.

This dataset has 4870 entries so CLT does apply.

### Q2. What are the null and alternate hypotheses?

Null hypothesis H0: Race does not have a significant impact on the rate of callbacks for resumes
Formally, H0: µ1 - µ2 = 0

Alternate hypotheses Ha: Race has a significant impact on the rate of callbacks for resumes
Formally, H1: µ1 - µ2 != 0

In [7]:
# Calculate the sample means
white_data = data[data.race=='w']
black_data = data[data.race=='b']
white_mean = white_data['call'].mean()
black_mean = black_data['call'].mean()
print('Sample mean(µ1) of call backs for white is ',white_mean)
print('Sample mean(µ2) of call backs for black is ',black_mean)

Sample mean(µ1) of call backs for white is  0.09650924056768417
Sample mean(µ2) of call backs for black is  0.0644763857126236


In [8]:
# Difference between calculated means for each type (w and b) of call backs
print('Difference between calculated means (µ1 - µ2) is %.3f ' %(white_mean-black_mean))

Difference between calculated means (µ1 - µ2) is 0.032 


### Q3. Compute margin of error, confidence interval, and p-value.

In [9]:
# Calculate Standard daviation of proportions of success of both black and white call backs
std_bw = np.sqrt(black_mean*(1-black_mean)/len(black_data)+white_mean*(1-white_mean)/len(white_data))
print('Standard daviation of proportions of success of both black and white call backs is %.5f' %std_bw)

Standard daviation of proportions of success of both black and white call backs is 0.00778


In [10]:
# Calculate margin of error 
margin_err = std_bw* 1.96 ## (z-score for desired 95% confidence level is 1.96)
print('Margin of error is %.6f' %margin_err)

Margin of error is 0.015255


In [11]:
# Calculate confidence interval 
conf_int = [0-margin_err, 0+margin_err] ## (As per our Null hypothesis, sample mean difference is zero)
print('confidence interval is ',conf_int)

confidence interval is  [-0.015255406348684322, 0.015255406348684322]


In [12]:
# Calculate z_statistics
observed_meanwb = white_mean-black_mean
expected_meanwb = 0    ## (As per our null hypothesis sample mean difference is zero)
SE = std_bw
z_stat = (observed_meanwb-expected_meanwb) / SE
print('z_stat is ',z_stat)

z_stat is  4.115550519002299


In [13]:
# Calculate p-value
p_value = stats.norm.sf(abs(z_stat))*2
print('p_value is ', p_value)

p_value is  3.86256381290969e-05


### Q4. Write a story describing the statistical significance in the context or the original problem.

The extremely low p-value indicates that we can reject the null hypothesis that there is no difference in the true rates of call backs for black sounding and white sounding namesvand it can be concluded that there is a perceived bias against black sounding names.

### Q5 Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

The analysis was carried out by taking race into consideration but this does not mean that race is necessarily the most important factor to callback success.

To identify other important deciding factors for the call back we can conduct principal component analysis (a method of factor analysis)
(Reference: https://en.wikipedia.org/wiki/Principal_component_analysis)

The most important factors can also be determined by calculating the means of different data values and the variable which yields the greatest difference can be considered as the important factor.
